In [46]:
## Load TEDS-D 2017 Data into a Pandas Data Frame
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import os

## Specify WD
wd = "/home/abf/BINF667013_Final_Project/"
os.chdir(wd)

teds=pd.read_csv("datasets/tedsd_puf_2017.csv")
teds

## Convert '-9' to NA
teds=teds.replace({-9:np.nan})

### Count the number of complete cases in this dataset

In [ ]:
teds.dropna().shape

### Count the number of cases with valid observations for at least 50% of available measures

In [ ]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 39, axis=1)
teds[mask].shape

### Count the number of cases with valid observations for at least 75% of available measures

In [38]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 19, axis=1)
teds[mask].shape

(1308896, 76)

### Count the number of cases with valid observations for at least 90% of available measures

In [21]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 14, axis=1)
teds[mask].shape

(759598, 76)

### Explore Correlation between missing values

In [ ]:
import missingno as msno
msno.heatmap(teds, fontsize=12)

In [ ]:
msno.dendrogram(teds)

### Restrict data to records that are at least 75% complete

In [48]:
mask=teds.apply(lambda x: np.isnan(x)).apply(lambda x: sum(x) < 19, axis=1)
print(teds[mask].shape)
teds = teds[mask]

## make a subset for development
#teds=teds.iloc[0:16300, ]

(1308896, 76)


### Convert Observations to Categorical Variables

In [ ]:
for col in teds.columns:
    if not (col =="CASEID" or col == "DISYR"):
        teds[col] = teds[col].astype('category')

### Use MissForest implemented in missingpy to impute missing values

In [43]:
## Stupid hack due to broken module name #WastingMyTime #SloppyCode
import sys
import sklearn.neighbors
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from os.path import exists

def random_chunks(df, n=100000, state=123):
    chunks = []
    i = 0
    while df.shape[0] > n:
        chunks.append(df.sample(n=n, replace=False, random_state=state))
        df = df.drop(chunks[i].index)
        i += 1

    chunks.append(df)
    return(chunks)
        
        
def impute_missforest(df, seed=123):
    print(df.index)
    imputer = MissForest(
        random_state=seed,
        criterion=('squared_error', 'gini'), # Package mispecifies criterion
        n_jobs=-1
    )
    X=imputer.fit_transform(
        df.drop(columns=['CASEID', 'DISYR']),
        cat_vars=np.array([x for x in range(0,74)])
    )

    dg=pd.DataFrame(X)
    dg.columns=df.columns[2:76]
    return(dg)

### Verfiy that All Chunks have at least 1000 observations for all variables

In [49]:
chunks = random_chunks(df=teds, n=100000)
for c in chunks:
    x=c.shape[0] - c.apply(lambda x: sum(np.isnan(x)), axis=0)
    print(c.index)
    print(x.loc[ x < 1000,])


Int64Index([ 795871,  901284,  335690,    3406,  545282,  312273, 1488949,
             479062, 1091987,  524842,
            ...
            1259270,  674537, 1245456, 1620326,  661097,  289106, 1419085,
            1079347,  888693,  725560],
           dtype='int64', length=100000)
Series([], dtype: int64)
Int64Index([ 797190,  312386,  141554,  520119,  892161,  972870,  721221,
             233771,  594648,  635180,
            ...
            1306806, 1393531,  876838,  768947, 1354991, 1325157,  546084,
             787119,  150578, 1228550],
           dtype='int64', length=100000)
Series([], dtype: int64)
Int64Index([1602400, 1535501,  395046, 1437636, 1219540,  589864,  452987,
             704548, 1127721,  283907,
            ...
            1494492,  509549,  869087, 1010916, 1113646,  887086,  671245,
             656525,  656087,  633822],
           dtype='int64', length=100000)
Series([], dtype: int64)
Int64Index([ 784533,  771040, 1026160,  651404,  778858, 1160798,  

In [50]:
pd.concat(chunks).shape

(1308896, 76)

In [ ]:
if exists("analysis_objects/teds_imputed.csv"):
    teds_imputed=pd.read_csv("analysis_objects/teds_imputed.csv")
else:
    chunks = random_chunks(df=teds, n=100000)
    dflist = [impute_missforest(c) for c in chunks]
    teds_imputed = pd.concat(dflist)
    teds_imputed.to_csv(
        path_or_buf="analysis_objects/teds_imputed.csv"
    )